# Ghost Convolution (Ghost Module) — Teori ve Uygulama

Bu defterde **Ghost Convolution / Ghost Module** yapısını ayrıntılı şekilde inceleyeceksin:

- Ghost Convolution nedir, hangi problemden doğmuştur?
- "Feature redundancy" (özellik fazlalığı) fikri nedir?
- Ghost Module nasıl çalışır: primary features + cheap operations
- Normal Conv ile parametre ve FLOPs sezgisel karşılaştırma
- PyTorch ile basit bir `GhostConv` implementasyonu
- Normal 1×1 Conv ile GhostConv çıktılarının ve parametrelerinin karşılaştırılması

Hedef, GhostConv'u sadece "yeni bir conv tipi" gibi değil,
**neden var, hangi tasarım kararıyla geldi, nasıl optimize ediyor** seviyesinde anlamaktır.


## 1. Motivasyon: Neden Ghost Convolution?

Klasik bir konvolüsyon düşünelim:

- Girdi: `(N, C_in, H, W)`
- Çıktı: `(N, C_out, H, W)`
- Parametre: `k × k × C_in × C_out`

Özellikle `C_out` büyük olduğunda hem **parametre sayısı** hem **FLOPs** maliyeti artar.

### Gözlem (GhostNet Makalesi):

- Konvolüsyon çıktısındaki feature map'lere bakıldığında,
- Birçok kanal **yüksek derecede benzer** (redundant),
- Yani "hem pahalı hem çok sayıda" feature üretmek gereksiz olabilir.

Bu gözleme dayanarak soru şudur:

> Tüm `C_out` feature map'lerini pahalı konvolüsyonla üretmek zorunda mıyım?
> Yoksa az sayıda "gerçek" feature üretip,
> geri kalanını ucuz işlemlerle türetebilir miyim?


## 2. Ghost Module Temel Fikri

Ghost Module, `C_in → C_out` dönüşümünü iki aşamada yapar:

1. **Primary Convolution (Gerçek feature'lar)**  
   - Girdi: `(N, C_in, H, W)`  
   - Çıkış: `(N, C_int, H, W)`  
   - Burada `C_int < C_out`. Örneğin `C_int = C_out / ratio`.  
   - Bu kısım klasik konvolüsyondur (pahalı kısım).

2. **Cheap Operations (Ghost feature'lar)**  
   - Girdi: `(N, C_int, H, W)`  
   - Ucuz işlemlerle yeni feature map'ler türetilir: `(N, C_ghost, H, W)`  
   - Örnek ucuz işlemler:
     - Depthwise 3×3 conv
     - Pixel shift
     - Küçük kernel conv
   - Amaç: feature'ların geri kalanını çok düşük maliyetle üretmek.

3. **Concat + Slice**  
   - `[primary_features, ghost_features]` birleştirilir:  
     `(N, C_int + C_ghost, H, W)`  
   - Gerekirse `C_out` kanal sayısına kırpılarak son çıktı elde edilir.

Bu sayede:

- Pahalı konvolüsyon **daha az kanala** uygulanır,
- Çıktının geri kalanı "ghost" (ucuzdan üretilmiş) feature'lardır.


## 3. Parametre / FLOPs Açısından Sezgi

Klasik 1×1 Conv için parametre sayısı:

\[
\text{Param}_{1x1} = C_{in} \times C_{out}
\]

GhostConv için ise:

- Primary Conv (1×1):  
  \( C_{in} \times C_{int} \)

- Cheap Ops (örneğin depthwise 3×3):  
  \( k \times k \times C_{int} \)  (k=3 için `9 × C_int`)

Toplam yaklaşık:

\[
\text{Param}_{Ghost} \approx C_{in} C_{int} + 9 C_{int}
\]

Eğer `C_int = C_out / r` seçilirse (örneğin r = 2):

- `C_int = C_out / 2`  
- Pahalı kısım artık `C_in × (C_out/2)` kadar.
- Kalan feature'lar ucuz depthwise'tan gelir.

Sonuç:

- \( C_{in} C_{out} \) yerine yaklaşık \( C_{in} C_{out} / r \) civarı maliyet,
- Özellikle `C_out` büyük olduğunda fark oldukça anlamlıdır.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GhostConv(nn.Module):
    """
    Basit Ghost Convolution implementasyonu.

    Adımlar:
    - Primary conv: C_in -> C_int  (daha az kanal, pahalı kısım)
    - Cheap operation (depthwise conv): C_int -> C_ghost
    - Concat primary ve ghost, sonra C_out'e kırp
    """
    def __init__(self, c_in, c_out, kernel_size=1, ratio=2, dw_kernel_size=3,
                 stride=1, padding=0):
        super().__init__()
        self.c_out = c_out
        self.ratio = ratio

        # Dahili kanal sayısı: kaç tane "gerçek" feature üreteceğiz?
        c_int = int(round(c_out / ratio))
        self.c_int = c_int
        c_ghost = c_out - c_int

        # 1) Primary conv (pahalı kısım)
        self.primary_conv = nn.Sequential(
            nn.Conv2d(
                in_channels=c_in,
                out_channels=c_int,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                bias=False
            ),
            nn.BatchNorm2d(c_int),
            nn.ReLU(inplace=True)
        )

        # 2) Cheap operation (ghost feature üretimi) - depthwise conv
        if c_ghost > 0:
            self.cheap_conv = nn.Sequential(
                nn.Conv2d(
                    in_channels=c_int,
                    out_channels=c_ghost,
                    kernel_size=dw_kernel_size,
                    stride=1,
                    padding=dw_kernel_size // 2,
                    groups=c_int,   # depthwise
                    bias=False
                ),
                nn.BatchNorm2d(c_ghost),
                nn.ReLU(inplace=True)
            )
        else:
            self.cheap_conv = None

    def forward(self, x):
        # 1) Temel feature'ları üret
        x_primary = self.primary_conv(x)   # (N, c_int, H, W)

        # 2) Ghost feature'ları üret
        if self.cheap_conv is not None:
            x_ghost = self.cheap_conv(x_primary)  # (N, c_ghost, H, W)
            out = torch.cat([x_primary, x_ghost], dim=1)  # (N, c_int + c_ghost, H, W)
        else:
            out = x_primary

        # 3) Fazla kanal varsa kırp (güvenlik için)
        if out.size(1) > self.c_out:
            out = out[:, :self.c_out, :, :]

        return out

# Küçük bir şekil testi
x = torch.randn(1, 32, 32, 32)
ghost = GhostConv(c_in=32, c_out=64, kernel_size=1, ratio=2, dw_kernel_size=3)

y = ghost(x)
print("Girdi şekli :", x.shape)
print("GhostConv çıkışı:", y.shape)


Girdi şekli : torch.Size([1, 32, 32, 32])
GhostConv çıkışı: torch.Size([1, 64, 32, 32])


## 4. Normal 1×1 Conv ile GhostConv Karşılaştırması

Şimdi aynı giriş/çıkış kanal sayıları için:

- Normal 1×1 Conv
- GhostConv

arasındaki parametre sayısını karşılaştıralım.


In [2]:
def count_params(m):
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

C_in, C_out = 32, 64
x = torch.randn(1, C_in, 32, 32)

std_conv = nn.Conv2d(C_in, C_out, kernel_size=1, bias=False)
ghost_conv = GhostConv(C_in, C_out, kernel_size=1, ratio=2, dw_kernel_size=3)

y_std = std_conv(x)
y_ghost = ghost_conv(x)

print("Normal 1x1 Conv parametre sayısı:", count_params(std_conv))
print("GhostConv parametre sayısı      :", count_params(ghost_conv))
print("Normal Conv çıktı şekli         :", y_std.shape)
print("GhostConv çıktı şekli           :", y_ghost.shape)


Normal 1x1 Conv parametre sayısı: 2048
GhostConv parametre sayısı      : 1440
Normal Conv çıktı şekli         : torch.Size([1, 64, 32, 32])
GhostConv çıktı şekli           : torch.Size([1, 64, 32, 32])


## 5. GhostConv Nerede Kullanılır?

Ghost Module, özellikle şu senaryolarda tercih edilir:

- **GhostNet** gibi hafif (lightweight) mimarilerde,
- Mobil / edge cihazlarda,
- Yüksek kanal sayılı katmanlarda:
  - Normal conv ile parametre ve FLOPs çok yüksek,
  - GhostConv ile benzer temsil gücü daha ucuza elde edilebilir.

Genel olarak GhostConv, klasik conv'un doğrudan yerine geçecek bir yapı değildir;
özellikle performans / maliyet optimizasyonu hedeflendiğinde anlamlı hale gelir.

**Özellikle:**
- 1×1 Conv yerine GhostConv kullanmak,
- Bazı 3×3 Conv katmanlarını GhostConv ile değiştirmek,
büyük mimarilerde ciddi parametre tasarrufu sağlar.


## 6. Kısa Özet

- GhostConvolution, feature map'lerin önemli bir kısmının **redundant** olduğu fikrinden doğmuştur.
- Fikir: 
  - Az sayıda "gerçek" feature'ı pahalı bir conv ile üret,
  - Geri kalan feature'ları bu temel feature'lardan **ucuz işlemlerle** türet.
- Yapı:
  - Primary Conv: `C_in → C_int`
  - Cheap Ops (depthwise conv vb.): `C_int → C_ghost`
  - Concat + slice: `C_int + C_ghost → C_out`
- Sonuç:
  - Normal Conv'e göre daha az parametre ve FLOPs,
  - Benzer (veya yeterli) temsil gücü.
- PyTorch'ta GhostConv yazmak için temel kalıp bu defterde verilmiştir;
  bunu kendi backbone'larına entegre ederek pratikte deneyebilirsin.

Bu noktadan sonra,
aynı fikri tam bir CNN modeli üzerinde kullanmak (örneğin bazı 1x1 conv katmanlarını GhostConv ile değiştirmek),
Ghost Module'ü gerçekten içselleştirmenin bir sonraki adımı olacaktır.
